# Threat Intelligence Analysis

In [ ]:
from ollama import Client
import os

In [ ]:
def call_llm(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity SOC analyst with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    options = {
        "temperature": 0.7
    }
    # Make the API call

    client = Client(host=os.getenv("OLLAMA_CHAT"))
    response = client.chat(
        model="llama3",
        messages=messages,
        options=options,
        stream=False            
    )
    return response['message']['content']

In [ ]:
def analyze_threat_data(file_path):
    # Read the raw threat data from the provided file
    with open(file_path, 'r') as file:
        raw_data = file.read()
    # Query LLM to identify and categorize potential threats
    identified_threats = call_llm(f"Analyze the following threat data and identify potential threats: {raw_data}")
    # Extract IoCs from the threat data
    extracted_iocs = call_llm(f"Extract all indicators of compromise (IoCs) from the following threat data: {raw_data}")
    # Obtain a detailed context or narrative behind the identified threats
    threat_context = call_llm(f"Provide a detailed context or narrative behind the identified threats in this data: {raw_data}")
    # Print the results
    print("Identified Threats:", identified_threats)
    print("\nExtracted IoCs:", extracted_iocs)
    print("\nThreat Context:", threat_context)

In [ ]:
if __name__ == "__main__":
    file_path = input("Enter the path to the raw threat data .txt file: ")
    analyze_threat_data(file_path)

# Log Analysis

In [ ]:
import asyncio
from ollama import Client
import os
import socket
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [ ]:
def call_llm(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity SOC analyst with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    options = {
        "temperature": 0.7
    }
    # Make the API call

    client = Client(host=os.getenv("OLLAMA_CHAT"))
    response = client.chat(
        model="llama3",
        messages=messages,
        options=options,
        stream=False            
    )
    return response['message']['content'].strip()

In [ ]:
async def handle_syslog():
    UDP_IP = "127.0.0.1"
    UDP_PORT = 5140

    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((UDP_IP, UDP_PORT))

    while True:
        data, addr = sock.recvfrom(1024)
        log_entry = data.decode('utf-8')
        analysis_result = call_llm(f"Analyze the following log entry for potential threats: {log_entry} \n\nIf you believe there may be suspicious activity, start your response with 'Suspicious Activity: ' and then your analysis. Provide nothing else. If you believe the behavior is normal, start your response with 'Normal Activity: ' and then your analysis. Provide nothing else.")

        if "Suspicious Activity" in analysis_result:
            print(f"{analysis_result}\n\n")

        await asyncio.sleep(0.1)  # A small delay to allow other tasks to run

In [ ]:
class Handler(FileSystemEventHandler):
    def process(self, event):
        if event.is_directory:
            return
        elif event.event_type == 'created':
            print(f"Received file: {event.src_path}")
            with open(event.src_path, 'r') as file:
                for line in file:
                    analysis_result = call_llm(f"Analyze the following log entry for potential threats: {line.strip()} \n\nIf you believe there may be suspicious activity, start your response with 'Suspicious Activity: ' and then your analysis. Provide nothing else.")
        if "Suspicious Activity" in analysis_result:
            print(f"Alert: {analysis_result}")
    def on_created(self, event):
        self.process(event)

In [ ]:
class Watcher:
    DIRECTORY_TO_WATCH = "/tmp/logs"
    def __init__(self):
        self.observer = Observer()
    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler,
          self.DIRECTORY_TO_WATCH, recursive=False)
        self.observer.start()
        try:
            while True:
                pass
        except:
            self.observer.stop()
            print("Observer stopped")

In [ ]:
if __name__ == "__main__":
    # The syslog watcher does not work in a Jupyter notebook as implemented
    #asyncio.run(handle_syslog()) 
    await handle_syslog()
    w = Watcher()
    w.run()

# System Analysis

In [ ]:
from ollama import Client
import os
import subprocess

In [ ]:
def call_llm(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity SOC analyst with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    options = {
        "temperature": 0.7
    }
    # Make the API call

    client = Client(host=os.getenv("OLLAMA_CHAT"))
    response = client.chat(
        model="llama3",
        messages=messages,
        options=options,
        stream=False            
    )
    return response['message']['content'].strip()

In [ ]:
# Function to run a command and return its output
def run_command(command):
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)
    return result.stdout

In [ ]:
# Windows
# Gather data from key locations
# registry_data = run_command('reg query HKLM /s')  # This produces MASSIVE data. Replace with specific registry keys if needed
# print(registry_data)
process_data = run_command('tasklist /v')
print(process_data)
network_data = run_command('netstat -an')
print(network_data)
scheduled_tasks = run_command('schtasks /query /fo LIST')
print(scheduled_tasks)
security_logs = run_command('wevtutil qe Security /c:10 /rd:true /f:text')  # Last 10 security events. Adjust as needed
print(security_logs)

In [ ]:
# Nix
# Gather data from key locations
process_data = run_command('ps aux')
print(process_data)
network_data = run_command('ss -tlnp')
print(network_data)
scheduled_tasks = run_command('crontab -l')
print(scheduled_tasks)
security_logs = run_command('tail -n 10 /var/log/syslog')  # Last 10 syslog events. Adjust as needed
print(security_logs)

In [ ]:
# Analyze the gathered data using ChatGPT
analysis_result = call_llm(f"Analyze the following system data for signs of APTs:\nProcess Data:\n{process_data}\n\nNetwork Data:\n{network_data}\n\nScheduled Tasks:\n{scheduled_tasks}\n\nSecurity Logs:\n{security_logs}") # Add Registry Data:\n{#registry_data}\n\n if used

# Display the analysis result
print(f"Analysis Result:\n{analysis_result}")

# Network Analysis

In [ ]:
from ollama import Client
import os
from scapy.all import rdpcap, IP, TCP

In [ ]:
def call_llm(prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity SOC analyst with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    options = {
        "temperature": 0.7
    }
    # Make the API call

    client = Client(host=os.getenv("OLLAMA_CHAT"))
    response = client.chat(
        model="llama3",
        messages=messages,
        options=options,
        stream=False            
    )
    return response['message']['content'].strip()

In [ ]:
# Read PCAP file
packets = rdpcap('example.pcap')

# Summarize the traffic (simplified example)
ip_summary = {}
port_summary = {}
protocol_summary = {}
for packet in packets:
    if packet.haslayer(IP):
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        ip_summary[f"{ip_src} to {ip_dst}"] = ip_summary.get(f"{ip_src} to {ip_dst}", 0) + 1

    if packet.haslayer(TCP):
        port_summary[packet[TCP].sport] = port_summary.get(packet[TCP].sport, 0) + 1

    if packet.haslayer(IP):
        protocol_summary[packet[IP].proto] = protocol_summary.get(packet[IP].proto, 0) + 1

# Create summary strings
ip_summary_str = "\n".join(f"{k}: {v} packets" for k, v in ip_summary.items())
port_summary_str = "\n".join(f"Port {k}: {v} packets" for k, v in port_summary.items())
protocol_summary_str = "\n".join(f"Protocol {k}: {v} packets" for k, v in protocol_summary.items())

# Combine summaries
total_summary = f"IP Summary:\n{ip_summary_str}\n\nPort Summary:\n{port_summary_str}\n\nProtocol Summary:\n{protocol_summary_str}"

In [ ]:
print(ip_summary_str)
print(port_summary_str)
print(protocol_summary_str)
print(total_summary)

In [ ]:
# Analyze using LLM
analysis_result = call_llm(f"Analyze the following summarized network traffic for anomalies or potential threats:\n{total_summary}")

# Print the analysis result
print(f"Analysis Result:\n{analysis_result}")